In [1]:
import pandas as pd
from sklearn import datasets
import statsmodels.api as sm
from estimagic_stargazer.stargazer import Stargazer 
from IPython.core.display import HTML
from tabulate import tabulate
from collections import namedtuple
from copy import deepcopy, copy
import numpy as np

diabetes = datasets.load_diabetes()
df = pd.DataFrame(diabetes.data)
df.columns = ['Age', 'Sex', 'BMI', 'ABP', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']
df['target'] = diabetes.target

est = sm.OLS(endog=df['target'], exog=sm.add_constant(df[df.columns[0:4]])).fit()
est2 = sm.OLS(endog=df['target'], exog=sm.add_constant(df[df.columns[0:6]])).fit()



C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2223: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [2]:
def extract_params_from_sm(model):
    to_concat = []
    params_list = ["params", "pvalues", "bse"]
    for col in params_list:
        to_concat.append(getattr(model, col))
    to_concat.append(model.conf_int()[0])
    to_concat.append(model.conf_int()[1])
    params_df = pd.concat(to_concat, axis=1)
    params_df.columns = ["value", "pvalue", "standard_error", "ci_lower", "ci_upper"]
    return params_df


def extract_info_from_sm(model):
    info = {}
    key_values = [
        "rsquared",
        "rsquared_adj",
        "scale",
        "fvalue",
        "f_pvalue",
        "df_model",
        "df_resid",
    ]
    for kv in key_values:
        info[kv] = getattr(model, kv)
    info["dependent_variable"] = model.model.endog_names

    return info


In [3]:
# use the functions defined above to create estimation result that is namedtuple
namedtuplee = namedtuple("namedtuplee", "params info")
est3 = namedtuplee(
params=extract_params_from_sm(est2),
info={**extract_info_from_sm(est2)},
)
del est3.info["df_model"]
del est3.info["df_resid"]


In [4]:
# using est3, create dictionary
est4 = {}
est4['params'] = deepcopy(est3.params)
est4['info'] = deepcopy(est3.info)


In [5]:
del est4['info']['rsquared'], est4['info']['rsquared_adj'],est4['info']['scale'], est4['info']['fvalue'], est4['info']['f_pvalue']

In [6]:
# using est3, create dictionary
est5 = {}
est5['params'] = est3.params
est5['info'] = est3.info


In [7]:
# check that code runs for non-list inputs
stargazer2=Stargazer(est3)

In [8]:
HTML(stargazer2.render_html())


In [9]:
# check stargazer works for sns regression result, dictionary and namedtuple
stargazer=Stargazer([est3,est4,est])

In [10]:
HTML(stargazer.render_html())


In [11]:
with open('./tex_tables/table0.tex','w') as f:
    f.write(stargazer.render_latex())

In [12]:
stargazer.title('Diabetes Study')

In [13]:
HTML(stargazer.render_html())


In [14]:
with open('./tex_tables/table1.tex','w') as f:
    f.write(stargazer.render_latex())


In [15]:
stargazer.custom_columns(['Model 1','Model 2','Model3'],[1,1,1])

In [16]:
HTML(stargazer.render_html())

In [17]:
with open('./tex_tables/table2.tex','w') as f:
    f.write(stargazer.render_latex())

In [18]:
stargazer.custom_columns('Test model name')

In [19]:
HTML(stargazer.render_html())

In [20]:
with open('./tex_tables/table3.tex','w') as f:
    f.write(stargazer.render_latex())

In [21]:
stargazer.show_model_numbers(False)

In [22]:
HTML(stargazer.render_html())

In [23]:
with open('./tex_tables/table4.tex','w') as f:
    f.write(stargazer.render_latex())

In [24]:
stargazer.significant_digits(2)

In [25]:
HTML(stargazer.render_html())

In [26]:
with open('./tex_tables/table5.tex','w') as f:
    f.write(stargazer.render_latex())

In [27]:
stargazer.show_confidence_intervals(True)

In [28]:
HTML(stargazer.render_html())

In [29]:
with open('./tex_tables/table6.tex','w') as f:
    f.write(stargazer.render_latex())

In [30]:
stargazer.covariate_order(['BMI', 'Age', 'S1', 'Sex'])

In [31]:
HTML(stargazer.render_html())

In [32]:
with open('./tex_tables/table7.tex','w') as f:
    f.write(stargazer.render_latex())

In [33]:
stargazer.rename_covariates({'Age': 'Oldness'})

In [34]:
HTML(stargazer.render_html())

In [35]:
with open('./tex_tables/table8.tex','w') as f:
    f.write(stargazer.render_latex())

In [36]:
stargazer.show_degrees_of_freedom(False)

In [37]:
HTML(stargazer.render_html())

In [38]:
with open('./tex_tables/table9.tex','w') as f:
    f.write(stargazer.render_latex())

In [39]:
stargazer.add_custom_notes(['First note', 'Second note'])

In [40]:
HTML(stargazer.render_html())

In [41]:
with open('./tex_tables/table10.tex','w') as f:
    f.write(stargazer.render_latex())

In [42]:
stargazer.significance_levels([0.1, 0.05, 0.07])

In [43]:
HTML(stargazer.render_html())

In [44]:
with open('./tex_tables/table11.tex','w') as f:
    f.write(stargazer.render_latex())

In [45]:
stargazer.append_notes(False)

In [46]:
HTML(stargazer.render_html())

In [47]:
with open('./tex_tables/table12.tex','w') as f:
    f.write(stargazer.render_latex())

# Check the support of MultiIndex

In [48]:
numarray = np.zeros(7)

In [49]:
pd.MultiIndex.from_arrays([numarray, est5['params'].index.values])

MultiIndex(levels=[[0.0], ['ABP', 'Age', 'BMI', 'S1', 'S2', 'Sex', 'const']],
           codes=[[0, 0, 0, 0, 0, 0, 0], [6, 1, 5, 2, 0, 3, 4]])

In [50]:
est5['params'].index = pd.MultiIndex.from_arrays([numarray, est5['params'].index.values])

In [51]:
est5['params']

value         pvalue  standard_error    ci_lower    ci_upper
0.0 const  152.133484  6.764758e-193        2.853063  146.525981  157.740987
    Age     24.703695   7.058611e-01       65.411058 -103.857320  153.264710
    Sex    -82.862181   2.020225e-01       64.850570 -210.321594   44.597231
    BMI    789.742054   4.243330e-28       66.886785  658.280598  921.203509
    ABP    397.582705   3.604149e-08       70.869661  258.293174  536.872235
    S1     197.852376   1.696004e-01      143.811973  -84.800339  480.505090
    S2    -169.250629   2.363914e-01      142.744232 -449.804771  111.303513

In [52]:
stargazermi = Stargazer(est5)

In [53]:
HTML(stargazermi.render_html())

TypeError: can only concatenate str (not "tuple") to str